# Python API Homework - VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Load the csv exported in Part I to a DataFrame

In [2]:
# Input File (CSV)
input_data_file = "../WeatherPy/output_data/cities.csv"

# Reading the data from csv file and loading into dataframe
Weather_Data = pd.read_csv(input_data_file)
Weather_Data = Weather_Data[['City','Country','Cloudiness','Humidity','Lat','Lng','Max Temp','Wind Speed']]
Weather_Data.head()

,City,Country,Cloudiness,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ilebo,CD,100.0,98.0,-4.33,20.59,69.28,2.15
1,mataura,NZ,100.0,94.0,-46.19,168.86,46.99,10.00
2,waddan,LY,4.0,76.0,29.16,16.14,66.36,7.72
3,saskylakh,RU,93.0,94.0,71.97,114.09,12.47,13.44
4,rikitea,PF,100.0,60.0,-23.12,-134.97,69.15,9.56


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Dropping NaN values records
Weather_Data_DF = Weather_Data.dropna(how="any")
   
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '5px solid white',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Store latitude and longitude in locations
locations = Weather_Data_DF[['Lat', 'Lng']]
weights = Weather_Data_DF['Humidity']

# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100, point_radius = 2)
                                 

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='5px solid white', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Favourite Weather Conditiins

fav_temp = [70,71,72,73,74,75,76,77,78,79,80]

Ideal_Weather_Data = Weather_Data_DF.loc[(Weather_Data_DF['Max Temp'] >= 70) 
                                         & (Weather_Data_DF['Max Temp'] <= 80) 
                                         & (Weather_Data_DF['Wind Speed'] <= 10) 
                                         & (Weather_Data_DF['Cloudiness'] <= 1),:]

Ideal_Weather_Data.count()


City          14
Country       14
Cloudiness    14
Humidity      14
Lat           14
Lng           14
Max Temp      14
Wind Speed    14
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# creating a copy of ideal weather dataframe
hotel_df = Ideal_Weather_Data.copy()

# Creating a new column in dataframe
hotel_df['Hotel Name'] = ""
hotel_df.head()


,City,Country,Cloudiness,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
88,camopi,GF,1.0,89.0,3.17,-52.33,72.64,3.29,
212,beloha,MG,0.0,60.0,-25.17,45.06,74.23,6.00,
316,agadez,NE,1.0,83.0,16.97,7.99,75.20,7.12,
360,hobyo,SO,1.0,82.0,5.35,48.53,79.18,5.01,
379,augusto correa,BR,0.0,77.0,-1.02,-46.63,78.03,7.67,


In [29]:
# find the first hotel in each city

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "lodging",
    "key": g_key,
    "radius" : 5000
}

# use iterrows to iterate through pandas dataframe

for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    
    # add city coordinates to params dict
    params['location'] = f"{lat},{lng}"
    
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_name = response['results'][0]['name']
        #print(f"{row['City']} - {hotel_name}")
        hotel_df.loc[index, 'Hotel Name'] = hotel_name  
        
    except (KeyError, IndexError):
        pass
        #print("Missing field/result... skipping.")


hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != "", : ]

# Displaying hotel dataframe
hotel_df.head()

,City,Country,Cloudiness,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
316,agadez,NE,1.0,83.0,16.97,7.99,75.20,7.12,Auberge d'Azel
379,augusto correa,BR,0.0,77.0,-1.02,-46.63,78.03,7.67,Fazenda Bacuri
382,trinidad,BO,0.0,62.0,-14.83,-64.90,75.90,2.58,Hotel Piesta
400,truth or consequences,US,1.0,18.0,33.13,-107.25,73.40,9.17,Charles Motel
405,bhanpuri,IN,0.0,75.0,21.10,80.92,77.20,4.89,Bhandari Hotel


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [31]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '5px solid white',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

highest_humidity = hotel_df['Humidity'].max()

# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
locations1 = Weather_Data_DF[['Lat', 'Lng']]
weights = Weather_Data_DF['Humidity']

heat_layer = gmaps.heatmap_layer(locations1, weights=weights, dissipating=False, max_intensity=highest_humidity,
                                 point_radius = 2)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='5px solid white', height='600px', margin='0 auto 0 auto', padding='1px', wi…